In [ ]:
# calculation book wind potential
# turbine , wind speed , wind direction, hub height, air density, air viscosity


In [ ]:
import os

from datetime import date, datetime, timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from openmeteo_requests import Client as OpenMeteoClient

In [ ]:
## defining function for API call

In [ ]:
def download_openmeteo_historical_data(
    start_date: str, 
    end_date: str,
    latitude: float, 
    longitude: float, 
    variables: list[str],
) -> pd.DataFrame:
    """Downloads the historical data from the Open-Meteo API.

    Args:
        start_date: Start date in the format YYYY-MM-DD.
        end_date: End date in the format YYYY-MM-DD.
        latitude: Latitude of the location.
        longitude: Longitude of the location.
        variables: List of weather variables to download. 

    Raises:
        ValueError: If the requested weather variable is not available.

    Returns:
        The historical data.
    """
    # Setup the Open-Meteo API client with cache and retry on error
    openmeteo = OpenMeteoClient()

    available_variables = [
        "temperature_2m",
        "relative_humidity_2m",
        "dew_point_2m",
        "apparent_temperature",
        "precipitation",
        "rain",
        "snowfall",
        "snow_depth",
        "weather_code",
        "pressure_msl",
        "surface_pressure",
        "cloud_cover",
        "cloud_cover_low",
        "cloud_cover_mid",
        "cloud_cover_high",
        "et0_fao_evapotranspiration",
        "vapour_pressure_deficit",
        "wind_speed_10m",
        "wind_speed_100m",
        "wind_direction_10m",
        "wind_direction_100m",
        "wind_gusts_10m",
        "soil_temperature_0_to_7cm",
        "soil_temperature_7_to_28cm",
        "soil_temperature_28_to_100cm",
        "soil_temperature_100_to_255cm",
        "soil_moisture_0_to_7cm",
        "soil_moisture_7_to_28cm",
        "soil_moisture_28_to_100cm",
        "soil_moisture_100_to_255cm",
    ]
    if set(variables) - set(available_variables):
        raise ValueError(f"Invalid variable(s): {set(variables) - set(available_variables)}")

    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": variables,
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_data = {
        "timestamp": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left",
        )
    }
    for i, variable in enumerate(variables):
        hourly_data[variable] = hourly.Variables(i).ValuesAsNumpy()
    return pd.DataFrame(data=hourly_data)

In [ ]:
import numpy as np
from scipy.optimize import fsolve
import pandas as pd

# Function to represent the equation
def equation(zo, wind_speed_100m, wind_speed_10m):
    return wind_speed_100m - wind_speed_10m * (np.log(100 / zo) / np.log(10 / zo))

# Function to calculate z0 for each time series entry
def calculate_z0(wind_speed_10m_series, wind_speed_100m_series):
    z0_values = []
    
    for wind_speed_10m, wind_speed_100m in zip(wind_speed_10m_series, wind_speed_100m_series):
        initial_guess = 0.1  # Initial guess for z0
        z0_solution = fsolve(equation, initial_guess, args=(wind_speed_100m, wind_speed_10m))
        z0_values.append(z0_solution[0])
    
    return z0_values




In [ ]:
## defining function for wind speed calculation at 150m

def calculate_velocity_at_150m(wind_speed_100m):
    return wind_speed_100m * (np.log(150 / 0.001) / np.log(100 / 0.001))

In [ ]:
## hrs in the wind speed range 

In [ ]:

import pandas as pd

def analyze_wind_speeds(data, timestamp_col, wind_speed_col, cut_in_wind_speed, cut_out_wind_speed):
    """
    Analyze wind speed data to calculate the number of hours:
    1. Below cut-in wind speed
    2. Between cut-in and cut-out wind speeds
    3. Above cut-out wind speed
    
    Args:
    data (pd.DataFrame): The wind speed dataset.
    timestamp_col (str): The name of the timestamp column.
    wind_speed_col (str): The name of the wind speed column.
    cut_in_wind_speed (float): The cut-in wind speed (m/s).
    cut_out_wind_speed (float): The cut-out wind speed (m/s).
    
    Returns:
    dict: A dictionary containing the hours for each category by year.
    """
    
    # Ensure the timestamp column is in datetime format
    data[timestamp_col] = pd.to_datetime(data[timestamp_col])
    
    # Ensure the timestamp column is the index
    data.set_index(timestamp_col, inplace=True)
    
    # Create conditions based on wind speed
    below_cut_in = data[wind_speed_col] < cut_in_wind_speed
    between_cut_in_and_out = (data[wind_speed_col] >= cut_in_wind_speed) & (data[wind_speed_col] <= cut_out_wind_speed)
    above_cut_out = data[wind_speed_col] > cut_out_wind_speed
    
    # Add a 'year' column for grouping by year
    data['year'] = data.index.year
    
    # Group data by year and count the number of occurrences in each condition
    hours_below_cut_in = data[below_cut_in].groupby('year').size()
    hours_between_cut_in_and_out = data[between_cut_in_and_out].groupby('year').size()
    hours_above_cut_out = data[above_cut_out].groupby('year').size()
    
    # Return the results as a dictionary
    return {
        "hours_below_cut_in": hours_below_cut_in,
        "hours_between_cut_in_and_out": hours_between_cut_in_and_out,
        "hours_above_cut_out": hours_above_cut_out
    }
    print(result['hours_below_cut_in'])
    print(result['hours_between_cut_in_and_out'])
    print(result['hours_above_cut_out'])


In [ ]:
## Collection of Wind data at 100m

In [ ]:
# Belgium 
historical_wind_5159_0279_BE = download_openmeteo_historical_data(
    start_date="2015-01-01", 
    end_date="2023-12-31",
    latitude=51.59, 
    longitude=2.79, 
    variables=[
        "wind_speed_10m",
        "wind_speed_100m",
        "wind_direction_100m",
    ],
)

# Appling function for calculating wind speed at 150m
historical_wind_5159_0279_BE['wind_speed_150m'] = calculate_velocity_at_150m(historical_wind_5159_0279_BE['wind_speed_100m'])

historical_wind_5159_0279_BE.to_parquet("/work/Data/openmeteo/historical_wind_5159_0279_BE.parquet")
historical_wind_5159_0279_BE


Coordinates 51.564144134521484°N 2.7767693996429443°E
Elevation 0.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,timestamp,wind_speed_10m,wind_speed_100m,wind_direction_100m,wind_speed_150m
0,2015-01-01 00:00:00+00:00,25.120159,36.123981,209.886444,37.396206
1,2015-01-01 01:00:00+00:00,25.438021,35.501549,210.465469,36.751850
2,2015-01-01 02:00:00+00:00,26.674032,37.038036,205.319641,38.342449
3,2015-01-01 03:00:00+00:00,26.081993,36.890778,208.565735,38.190006
4,2015-01-01 04:00:00+00:00,23.166216,32.599388,226.789886,33.747482
...,...,...,...,...,...
78883,2023-12-31 19:00:00+00:00,58.361095,73.186325,228.389954,75.763817
78884,2023-12-31 20:00:00+00:00,61.905342,78.040321,230.803589,80.788765
78885,2023-12-31 21:00:00+00:00,61.102509,77.412560,226.507370,80.138893
78886,2023-12-31 22:00:00+00:00,58.321106,72.577286,233.471054,75.133331


In [ ]:
# Calculate z0 values based on the time series
historical_wind_5159_0279_BE['z0'] = calculate_z0(historical_wind_5159_0279_BE['wind_speed_10m'], historical_wind_5159_0279_BE['wind_speed_100m'])



/tmp/ipykernel_82/3907876558.py:7: RuntimeWarning: invalid value encountered in log
  return wind_speed_100m - wind_speed_10m * (np.log(100 / zo) / np.log(10 / zo))
/shared-libs/python3.9/py/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [ ]:
z0 = historical_wind_5159_0279_BE['z0']
z0

timestamp
2015-01-01 00:00:00+00:00    0.052137
2015-01-01 01:00:00+00:00    0.029666
2015-01-01 02:00:00+00:00    0.100000
2015-01-01 03:00:00+00:00    0.038633
2015-01-01 04:00:00+00:00    0.035009
                               ...   
2023-12-31 19:00:00+00:00    0.100000
2023-12-31 20:00:00+00:00    0.100000
2023-12-31 21:00:00+00:00    0.100000
2023-12-31 22:00:00+00:00    0.100000
2023-12-31 23:00:00+00:00    0.100000
Name: z0, Length: 78888, dtype: float64

In [ ]:
#denmark
historical_wind_5645_0807_DK = download_openmeteo_historical_data(
    start_date="2015-01-01", 
    end_date="2023-12-31",
    latitude=56.45, 
    longitude=8.07, 
    variables=[
        "wind_speed_100m",
        "wind_direction_100m",
    ],
)



# Appling function
historical_wind_5645_0807_DK['wind_speed_150m'] = calculate_velocity_at_150m(historical_wind_5645_0807_DK['wind_speed_100m'])
historical_wind_5645_0807_DK.to_parquet("/work/Data/openmeteo/historical_wind_5645_0807_DK.parquet")
historical_wind_5645_0807_DK

Coordinates 56.48506164550781°N 8.232848167419434°E
Elevation 0.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,timestamp,wind_speed_100m,wind_direction_100m,wind_speed_150m
0,2015-01-01 00:00:00+00:00,42.186100,242.560349,43.671822
1,2015-01-01 01:00:00+00:00,43.672913,240.899612,45.210995
2,2015-01-01 02:00:00+00:00,44.445057,234.893845,46.010334
3,2015-01-01 03:00:00+00:00,48.971905,231.866379,50.696609
4,2015-01-01 04:00:00+00:00,51.394333,230.400009,53.204350
...,...,...,...,...
78883,2023-12-31 19:00:00+00:00,37.705597,132.678452,39.033520
78884,2023-12-31 20:00:00+00:00,34.479279,139.658325,35.693577
78885,2023-12-31 21:00:00+00:00,29.627365,152.500946,30.670788
78886,2023-12-31 22:00:00+00:00,27.708309,155.432846,28.684147


In [ ]:
#Germany
historical_wind_5413_0716_DE = download_openmeteo_historical_data(
    start_date="2015-01-01", 
    end_date="2023-12-31",
    latitude=54.45, 
    longitude=7.16, 
    variables=[
        "wind_speed_100m",
        "wind_direction_100m",
    ],
)



# Appling function
historical_wind_5413_0716_DE['wind_speed_150m'] = calculate_velocity_at_150m(historical_wind_5413_0716_DE['wind_speed_100m'])
historical_wind_5413_0716_DE.to_parquet("/work/Data/openmeteo/historical_wind_5413_0716_DE.parquet")
historical_wind_5413_0716_DE

Coordinates 54.44639587402344°N 7.235294342041016°E
Elevation 0.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,timestamp,wind_speed_100m,wind_direction_100m,wind_speed_150m
0,2015-01-01 00:00:00+00:00,49.493126,246.418335,51.236187
1,2015-01-01 01:00:00+00:00,50.876041,243.072403,52.667805
2,2015-01-01 02:00:00+00:00,52.294258,239.374542,54.135971
3,2015-01-01 03:00:00+00:00,55.385036,242.935364,57.335602
4,2015-01-01 04:00:00+00:00,54.149796,238.317688,56.056854
...,...,...,...,...
78883,2023-12-31 19:00:00+00:00,52.085667,188.746078,53.920033
78884,2023-12-31 20:00:00+00:00,56.224575,189.582870,58.204704
78885,2023-12-31 21:00:00+00:00,54.170128,188.022308,56.077904
78886,2023-12-31 22:00:00+00:00,54.172523,184.573837,56.080383


In [ ]:
# Netherlands
historical_wind_5214_0345_NL = download_openmeteo_historical_data(
    start_date="2015-01-01", 
    end_date="2023-12-31",
    latitude=52.14, 
    longitude=3.45, 
    variables=[
        "wind_speed_100m",
        "wind_direction_100m",
    ],
)

# Applying function
historical_wind_5214_0345_NL['wind_speed_150m'] = calculate_velocity_at_150m(historical_wind_5214_0345_NL['wind_speed_100m'])
historical_wind_5214_0345_NL.to_parquet("/work/Data/openmeteo/historical_wind_5214_0345_NL.parquet")
historical_wind_5214_0345_NL


Coordinates 52.12653732299805°N 3.4806628227233887°E
Elevation 0.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,timestamp,wind_speed_100m,wind_direction_100m,wind_speed_150m
0,2015-01-01 00:00:00+00:00,48.877869,224.403259,50.599262
1,2015-01-01 01:00:00+00:00,48.877869,224.403259,50.599262
2,2015-01-01 02:00:00+00:00,48.407619,218.659821,50.112450
3,2015-01-01 03:00:00+00:00,47.954376,228.652313,49.643246
4,2015-01-01 04:00:00+00:00,45.940571,232.321838,47.558517
...,...,...,...,...
78883,2023-12-31 19:00:00+00:00,70.755333,220.253403,73.247208
78884,2023-12-31 20:00:00+00:00,65.433899,226.114471,67.738365
78885,2023-12-31 21:00:00+00:00,66.702080,225.874588,69.051208
78886,2023-12-31 22:00:00+00:00,65.425972,224.331284,67.730156


In [ ]:
import pandas as pd

# Load the dataset
historical_wind_5159_0279_BE_1 = historical_wind_5159_0279_BE

# Call the analyze_wind_speeds function
result = analyze_wind_speeds(
    data=historical_wind_5159_0279_BE_1,  # Your dataset
    timestamp_col='timestamp',            # Column name for timestamps (change if necessary)
    wind_speed_col='wind_speed_150m',     # Column name for wind speed at 150m (adjust if different)
    cut_in_wind_speed=3,                # Your cut-in wind speed (change as needed)
    cut_out_wind_speed=31               # Your cut-out wind speed (change as needed)
)
result = pd.DataFrame(result)
result.to_csv("/work/Data/data of Oprating hr /historical_wind_5159_0279_BE_oprating_hr.csv")
# Output the results
print("Hours below cut-in wind speed:")
print(result['hours_below_cut_in'])

print("\nHours between cut-in and cut-out wind speed:")
print(result['hours_between_cut_in_and_out'])

print("\nHours above cut-out wind speed:")
print(result['hours_above_cut_out'])


KeyError: 'timestamp'

In [ ]:

# Load the dataset
historical_wind_5645_0807_DK_1=historical_wind_5645_0807_DK

# Call the analyze_wind_speeds function
result = analyze_wind_speeds(
    data=historical_wind_5645_0807_DK_1,  # Your dataset
    timestamp_col='timestamp',          # Column name for timestamps (change if necessary)
    wind_speed_col='wind_speed_150m',   # Column name for wind speed at 150m (adjust if different)
    cut_in_wind_speed= 3,              # Your cut-in wind speed (change as needed)
    cut_out_wind_speed= 31             # Your cut-out wind speed (change as needed)
)
result = pd.DataFrame(result)
result.to_csv("/work/Data/data of Oprating hr /historical_wind_5645_0807_DK_oprating_hr.csv")
# Output the results
print("Hours below cut-in wind speed:")
print(result['hours_below_cut_in'])

print("\nHours between cut-in and cut-out wind speed:")
print(result['hours_between_cut_in_and_out'])

print("\nHours above cut-out wind speed:")
print(result['hours_above_cut_out'])

Hours below cut-in wind speed:
year
2015    22
2016    42
2017    28
2018    36
2019    27
2020    29
2021    31
2022    29
2023    50
Name: hours_below_cut_in, dtype: int64

Hours between cut-in and cut-out wind speed:
year
2015    3531
2016    3980
2017    4444
2018    4840
2019    4580
2020    4407
2021    5034
2022    4462
2023    4843
Name: hours_between_cut_in_and_out, dtype: int64

Hours above cut-out wind speed:
year
2015    5207
2016    4762
2017    4288
2018    3884
2019    4153
2020    4348
2021    3695
2022    4269
2023    3867
Name: hours_above_cut_out, dtype: int64


In [ ]:
# germany
# Load the dataset
historical_wind_5413_0716_DE_1=historical_wind_5413_0716_DE

# Call the analyze_wind_speeds function
result = analyze_wind_speeds(
    data=historical_wind_5413_0716_DE_1,  # Your dataset
    timestamp_col='timestamp',          # Column name for timestamps (change if necessary)
    wind_speed_col='wind_speed_150m',   # Column name for wind speed at 150m (adjust if different)
    cut_in_wind_speed= 3,              # Your cut-in wind speed (change as needed)
    cut_out_wind_speed= 31             # Your cut-out wind speed (change as needed)
)

result = pd.DataFrame(result)
result.to_csv("/work/Data/data of Oprating hr /historical_wind_5413_0716_DE_oprating_hr.csv")

# Output the results
print("Hours below cut-in wind speed:")
print(result['hours_below_cut_in'])

print("\nHours between cut-in and cut-out wind speed:")
print(result['hours_between_cut_in_and_out'])

print("\nHours above cut-out wind speed:")
print(result['hours_above_cut_out'])

Hours below cut-in wind speed:
year
2015    41
2016    46
2017    40
2018    49
2019    18
2020    53
2021    23
2022    31
2023    38
Name: hours_below_cut_in, dtype: int64

Hours between cut-in and cut-out wind speed:
year
2015    3272
2016    3827
2017    3372
2018    3382
2019    3206
2020    3342
2021    3856
2022    3417
2023    3691
Name: hours_between_cut_in_and_out, dtype: int64

Hours above cut-out wind speed:
year
2015    5447
2016    4911
2017    5348
2018    5329
2019    5536
2020    5389
2021    4881
2022    5312
2023    5031
Name: hours_above_cut_out, dtype: int64


In [ ]:
# netherlands
# Load the dataset
historical_wind_5214_0345_NL_1=historical_wind_5214_0345_NL

# Call the analyze_wind_speeds function
result = analyze_wind_speeds(
    data=historical_wind_5214_0345_NL_1,  # Your dataset
    timestamp_col='timestamp',          # Column name for timestamps (change if necessary)
    wind_speed_col='wind_speed_150m',   # Column name for wind speed at 150m (adjust if different)
    cut_in_wind_speed= 3,              # Your cut-in wind speed (change as needed)
    cut_out_wind_speed= 31             # Your cut-out wind speed (change as needed)
)

result = pd.DataFrame(result)
result.to_csv("/work/Data/data of Oprating hr /historical_wind_5214_0345_NL_oprating_hr.csv")

# Output the results
print("Hours below cut-in wind speed:")
print(result['hours_below_cut_in'])

print("\nHours between cut-in and cut-out wind speed:")
print(result['hours_between_cut_in_and_out'])

print("\nHours above cut-out wind speed:")
print(result['hours_above_cut_out'])

Hours below cut-in wind speed:
year
2015    24
2016    36
2017    50
2018    46
2019    46
2020    35
2021    33
2022    36
2023    31
Name: hours_below_cut_in, dtype: int64

Hours between cut-in and cut-out wind speed:
year
2015    3520
2016    4146
2017    3794
2018    3863
2019    3807
2020    3464
2021    4036
2022    4085
2023    3411
Name: hours_between_cut_in_and_out, dtype: int64

Hours above cut-out wind speed:
year
2015    5216
2016    4602
2017    4916
2018    4851
2019    4907
2020    5285
2021    4691
2022    4639
2023    5318
Name: hours_above_cut_out, dtype: int64


In [ ]:
result_hours_below_cut_in = result['hours_below_cut_in']
result_hours_between_cut_in_and_out = result['hours_between_cut_in_and_out']
result_hours_above_cut_out = result['hours_above_cut_out']


In [ ]:
result = pd.DataFrame({'hours_below_cut_in': result_hours_below_cut_in, 'hours_between_cut_in_and_out': result_hours_between_cut_in_and_out, 'hours_above_cut_out': result_hours_above_cut_out})
result.to_csv("/work/Data/openmeteo/result.csv", index=False)

In [ ]:
result.to_csv("/work/Data/openmeteo/result.csv", index=True)

In [ ]:
result

,hours_below_cut_in,hours_between_cut_in_and_out,hours_above_cut_out
year,,,
2015,24,3520,5216
2016,36,4146,4602
2017,50,3794,4916
2018,46,3863,4851
2019,46,3807,4907
2020,35,3464,5285
2021,33,4036,4691
2022,36,4085,4639
2023,31,3411,5318


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c6d312a4-2fad-4555-a64a-cb89f1604b41' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>